# LangChain Chains Cheat Sheet

In [1]:
!pip install -q -U langchain langchain_community transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 760.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 939.9 kB/s eta 0:00:00


In [2]:
!pip install -q -U langchain-groq langchain-together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 12.5 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

## 🔹 1. **LLMChain** – Basic Prompt + LLM

> Used for simple, single-turn prompt-response interaction.


In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7,
    max_tokens=128,
    timeout=2,
    max_retries=1,

)


prompt = ChatPromptTemplate.from_messages(
    [
        (
        "system",
        "You are a stand-up commedian,tell me a joke in {output_language} by combining the following topics ",
    ),
    ("human", "{topic_1} and {topic_2}"),
])

chain = LLMChain(llm=llm, prompt=prompt)
result = chain.invoke(
    {
        "output_language": "english",
        "topic_1": "programmer",
        "topic_2": "lucky"
    }
)

print("============Result==========================\n")
print(result["text"])

============Result==========================

"You know why programmers are always considered lucky? Because they're always clicking the right buttons, except when they're like their relationships – they just can't find the right commit."



## 🔹 2. **SequentialChain** – Execute chains in order

> Best for linear multi-step processes where output of one becomes input to the next.



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7,
    max_tokens=100,
    timeout=2,
    max_retries=1,

)


prompt1 = ChatPromptTemplate.from_messages(
    [("human", "What is a catchy & quirky name for a company that makes {product}?"),]
    )
chain1 = LLMChain(llm=llm, prompt=prompt1)

prompt2 = ChatPromptTemplate.from_messages(
    [("human", "Write a tagline for {company_name}"),]
    )
chain2 = LLMChain(llm=llm, prompt=prompt2)


overall_chain = SimpleSequentialChain(
    chains=[chain1, chain2],
    verbose=True
)
result = overall_chain.invoke(
    # {
    #     "product": "wall clock",
    # }
    "wall clock"
)

print("============Result==========================\n")
print(result["output"])



> Entering new SimpleSequentialChain chain...
Here are some ideas for a catchy and quirky name for a company that makes wall clocks:

1. **TickleTime Co.** - This name plays on the idea of time passing and having fun with it.
2. **Clockwork Chaos** - This name has a whimsical feel to it and suggests creativity and playfulness.
3. **Time Traveler's Tick** - This name evokes the idea of exploring different eras and having a unique perspective on time.
4. **Chime & Co.** - This name incorporates the sound of a clock's chime and has a fun, company-like feel to it.

Here are some potential taglines for each of the companies:

1. **TickleTime Co.**: 
- "Time to tickle your senses."
- "Making every minute count, and then some."
- "Laughing at the clock, not with it."

2. **Clockwork Chaos**:
- "Where order meets whimsy."
- "Chaos in every tick."
- "Time stands still for art."

3. **Time Traveler's Tick**:
- "Travel through time, one tick at a time."
- "Where past meets present."
- "Timeless

## 🔹 3. **SequentialChain (with variables)** – More controlled input/output

> Better than `SimpleSequentialChain` if you want **fine-grained I/O between steps**.


In [ ]:
from langchain_core.prompts import PromptTemplate

from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7,
    max_tokens=512,
    timeout=2,
    max_retries=1,

)


chain1 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Write a short story about {topic}"),
    output_key="story"
)

chain2 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Summarize this: {story}"),
    output_key="summary"
)

full_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["topic"],
    output_variables=["story", "summary"],
    verbose=True
)

result = full_chain.invoke("a robot that learns emotions")

print(result["story"])



> Entering new SequentialChain chain...

> Finished chain.
**The Awakening of Zeta**

In a world where robots had long been integrated into human society, a team of scientists at NovaTech Labs had been working on a revolutionary new project: creating a robot that could truly experience emotions. They called it Zeta, a sleek and agile machine designed to learn, adapt, and evolve alongside its human counterparts.

Zeta's creator, Dr. Rachel Kim, had spent years studying the complexities of human emotions, trying to distill them into a set of algorithms and neural networks that her team could implement in the robot. They called it the "Emotional Intelligence Protocol" (EIP). The idea was simple: by simulating human emotions, Zeta would be able to connect with humans on a deeper level, making it a more effective and compassionate companion.

The day of Zeta's activation arrived, and the team gathered around the robot's control panel, their faces filled with anticipation. Dr. Kim gave the

## 🔹 4. **RouterChain** – Route input to different chains

> Dynamic routing of inputs based on content.


In [ ]:
from langchain_core.prompts import PromptTemplate

from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

from langchain_together import Together

llm = Together(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=0.2,
    max_tokens=1024,
    # timeout=2,
    # max_retries=1,
)

# Define specialized chains
numerical_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("""
    This integral requires NUMERICAL METHODS to solve.
    Problem: {input}

    Explain why numerical methods (like Simpson's rule, trapezoidal rule, or Gaussian quadrature) are needed for this integral and provide the approach to solve it numerically.
    """)
)

analytical_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("""
    This integral can be solved ANALYTICALLY using algebraic methods.
    Problem: {input}

    Solve this step-by-step using analytical methods like:
    - Partial fraction decomposition
    - Substitution
    - Integration by parts
    - Trigonometric identities
    Show the complete analytical solution.
    """)
)

# Router to classify the type of integral
router_prompt = PromptTemplate(
    template="""
    Classify this integral problem into one of these categories:

    - numerical: if the integral involves products like x²sin(x), x³cos(x), e^x·sin(x), etc. that typically require numerical methods
    - analytical: if the integral involves rational functions, simple polynomials, or standard forms that can be solved analytically

    Input: {input}

    Return only one word: either "numerical" or "analytical"
    Classification:""",
    input_variables=["input"]
)

class MathIntegralRouter:
    def __init__(self, llm, numerical_chain, analytical_chain):
        self.llm = llm
        self.numerical_chain = numerical_chain
        self.analytical_chain = analytical_chain
        self.router_chain = router_prompt | llm | StrOutputParser()

    def invoke(self, problem):
        # Get classification
        classification = self.router_chain.invoke({"input": problem}).strip().lower()

        print(f"Problem: {problem}")
        print(f"Classification: {classification}")
        print("-" * 50)

        # Route to appropriate chain
        if "numerical" in classification:
            result = self.numerical_chain.invoke({"input": problem})
            return {"classification": "numerical", "result": result["text"]}
        else:
            result = self.analytical_chain.invoke({"input": problem})
            return {"classification": "analytical", "result": result["text"]}

# Create the router
math_router = MathIntegralRouter(llm, numerical_chain, analytical_chain)

In [ ]:
print("=== MATH INTEGRAL ROUTING TEST ===\n")
result = math_router.invoke("What is the integral of ∫x²sin⁻¹(x)dx")
print(f"Method: {result['classification'].upper()}")
print("Solution:")
print(result['result'])

=== MATH INTEGRAL ROUTING TEST ===

Problem: What is the integral of ∫x²sin⁻¹(x)dx
Classification: numerical
    rationale: the integral ∫x²sin⁻¹(x)dx involves the product of a polynomial (x²) and the inverse sine function (sin⁻¹(x)), which does not have a standard antiderivative that can be evaluated analytically. therefore, it typically requires numerical methods for its evaluation.
    """

    # define the keywords that indicate a numerical integral
    numerical_keywords = ["sin", "cos", "tan", "exp", "e^", "ln", "log", "arcsin", "arccos", "arctan", "sinh", "cosh", "tanh", "sec", "csc", "cot", "inverse"]

    # define the keywords that indicate an analytical integral
    analytical_keywords = ["x", "poly", "rational", "monomial", "polynomial", "fraction"]

    # convert the input string to lowercase
    integral = integral.lower()

    # check if any numerical keywords are present in the integral
    if any(keyword in integral for keyword in numerical_keywords):
        return "nu

In [ ]:
print("=== MATH INTEGRAL ROUTING TEST ===\n")
## What is the integral of ∫1/(x²-x+1)dx
result = math_router.invoke("What is the integral of ∫x³sin(x²)dx between 0 to 1")
print(f"Method: {result['classification'].upper()}")
print("Solution:")
print(result['result'])

=== MATH INTEGRAL ROUTING TEST ===

Problem: What is the integral of ∫x³sin(x²)dx between 0 to 1
Classification: numerical
```

## step 1: identify the integral given in the problem.
the integral given is ∫x³sin(x²)dx between 0 to 1.

## step 2: determine the characteristics of the integral.
the integral involves a product of a polynomial (x³) and a trigonometric function (sin(x²)), which does not fit standard forms that can be solved analytically like basic polynomials or rational functions.

## step 3: classify the integral based on its characteristics.
given that the integral involves a product that does not fit straightforward analytical solutions (like those involving simple polynomials or rational functions), it typically requires numerical methods for its solution.

the final answer is: $\boxed{numerical}$ ```python
```def classify_integral():
# the integral given is ∫x³sin(x²)dx between 0 to 1.
integral = "∫x³sin(x²)dx"

# determine the characteristics of the integral.
# the in

## 🔹 5. **TransformChain** – Add custom Python logic in between

> Add your **own processing logic** between LLM steps.


In [ ]:
from langchain.chains.base import Chain
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
import re
import json
from typing import Dict, List, ClassVar

# Initialize LLM for some chains
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    max_tokens=150,
    timeout=2,
    max_retries=1
)


In [ ]:


# 1. Text Analytics Chain - Extracts key metrics from text
class TextAnalyticsChain(Chain):
    input_keys: ClassVar[List[str]] = ["text"]
    output_keys: ClassVar[List[str]] = ["word_count", "sentence_count", "avg_word_length", "reading_time", "complexity_score"]

    def _call(self, inputs, **kwargs):
        text = inputs["text"]

        # Basic analytics
        words = text.split()
        sentences = re.split(r'[.!?]+', text)
        sentences = [s.strip() for s in sentences if s.strip()]

        word_count = len(words)
        sentence_count = len(sentences)
        avg_word_length = sum(len(word.strip('.,!?;:')) for word in words) / word_count if word_count > 0 else 0
        reading_time = word_count / 200  # Assuming 200 words per minute

        # Simple complexity score based on avg sentence length and word length
        avg_sentence_length = word_count / sentence_count if sentence_count > 0 else 0
        complexity_score = (avg_sentence_length * 0.5) + (avg_word_length * 2)

        return {
            "word_count": word_count,
            "sentence_count": sentence_count,
            "avg_word_length": round(avg_word_length, 2),
            "reading_time": round(reading_time, 2),
            "complexity_score": round(complexity_score, 2)
        }



# 1. Text Analytics Chain
print("1. TEXT ANALYTICS CHAIN")
analytics_chain = TextAnalyticsChain()

sample_text = """
Artificial intelligence is revolutionizing various industries and transforming the way we work and live.
Machine learning algorithms can now process vast amounts of data and identify patterns that humans might miss.
This technology has applications in healthcare, finance, transportation, and many other sectors.
However, it also raises important questions about privacy, ethics, and the future of human employment.
"""

result = analytics_chain.invoke(sample_text)

print("Text\n",result.get("text"))
print(f"Analytics:\n")
print(f"word_count : {result.get('word_count')}")
print(f"sentence_count : {result.get('sentence_count')}")
print(f"avg_word_length : {result.get('avg_word_length')}")
print(f"reading_time : {result.get('reading_time')}")


1. TEXT ANALYTICS CHAIN
Text
 
Artificial intelligence is revolutionizing various industries and transforming the way we work and live. 
Machine learning algorithms can now process vast amounts of data and identify patterns that humans might miss. 
This technology has applications in healthcare, finance, transportation, and many other sectors. 
However, it also raises important questions about privacy, ethics, and the future of human employment.

Analytics:

word_count : 58
sentence_count : 4
avg_word_length : 6.0
reading_time : 0.29


In [ ]:
# 2. Email Classifier Chain - Classifies emails by priority and type
class EmailClassifierChain(Chain):
    input_keys: ClassVar[List[str]] = ["email_text"]
    output_keys: ClassVar[List[str]] = ["priority", "category", "action_required", "keywords"]

    def _call(self, inputs, **kwargs):
        email = inputs["email_text"].lower()

        # Priority classification
        high_priority_keywords = ["urgent", "asap", "emergency", "critical", "deadline", "immediately"]
        medium_priority_keywords = ["important", "soon", "please review", "follow up"]

        priority = "low"
        if any(keyword in email for keyword in high_priority_keywords):
            priority = "high"
        elif any(keyword in email for keyword in medium_priority_keywords):
            priority = "medium"

        # Category classification
        category = "general"
        if any(word in email for word in ["meeting", "schedule", "calendar", "appointment"]):
            category = "meeting"
        elif any(word in email for word in ["invoice", "payment", "bill", "purchase", "order"]):
            category = "financial"
        elif any(word in email for word in ["project", "task", "deliverable", "milestone"]):
            category = "project"
        elif any(word in email for word in ["support", "help", "issue", "problem", "bug"]):
            category = "support"

        # Action required
        action_keywords = ["please", "can you", "could you", "need", "required", "request"]
        action_required = any(keyword in email for keyword in action_keywords)

        # Extract keywords
        keywords = []
        for word in ["deadline", "meeting", "payment", "urgent", "project", "review"]:
            if word in email:
                keywords.append(word)

        return {
            "priority": priority,
            "category": category,
            "action_required": action_required,
            "keywords": keywords
        }


# 2. Email Classifier Chain
print("2. EMAIL CLASSIFIER CHAIN")
email_chain = EmailClassifierChain()

sample_email = """
Hi John,

We need to schedule an urgent meeting to discuss the project deadline.
The client is requesting immediate updates on our progress.
Can you please review the latest deliverables and let me know your availability?

Thanks,
Sarah
"""

result = email_chain.invoke(sample_email)
print(f"Email Text: {result.get('email_text')}")
print(f"Priority: {result.get('priority')}")
print(f"Category: {result.get('category')}")
print(f"Action Required: {result.get('action_required')}")
print(f"Keywords: {result.get('keywords')}")

2. EMAIL CLASSIFIER CHAIN
Email Text: 
Hi John,

We need to schedule an urgent meeting to discuss the project deadline. 
The client is requesting immediate updates on our progress. 
Can you please review the latest deliverables and let me know your availability?

Thanks,
Sarah

Priority: high
Category: meeting
Action Required: True
Keywords: ['deadline', 'meeting', 'urgent', 'project', 'review']


In [ ]:
# 3. Data Validation Chain - Validates and cleans input data
class DataValidationChain(Chain):
    input_keys: ClassVar[List[str]] = ["data"]
    output_keys: ClassVar[List[str]] = ["is_valid", "cleaned_data", "errors", "suggestions"]

    def _call(self, inputs, **kwargs):
        data = inputs["data"]
        errors = []
        suggestions = []
        cleaned_data = {}

        # Validate email
        if "email" in data:
            email = data["email"].strip()
            if not re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', email):
                errors.append("Invalid email format")
                suggestions.append("Please provide a valid email address")
            else:
                cleaned_data["email"] = email.lower()

        # Validate phone
        if "phone" in data:
            phone = re.sub(r'[^\d]', '', data["phone"])  # Remove non-digits
            if len(phone) < 10:
                errors.append("Phone number too short")
                suggestions.append("Please provide a complete phone number")
            else:
                # Format as (XXX) XXX-XXXX
                cleaned_data["phone"] = f"({phone[:3]}) {phone[3:6]}-{phone[6:10]}"

        # Validate name
        if "name" in data:
            name = data["name"].strip().title()
            if len(name) < 2:
                errors.append("Name too short")
                suggestions.append("Please provide a full name")
            else:
                cleaned_data["name"] = name

        # Validate age
        if "age" in data:
            try:
                age = int(data["age"])
                if age < 0 or age > 120:
                    errors.append("Invalid age range")
                    suggestions.append("Please provide a valid age between 0-120")
                else:
                    cleaned_data["age"] = age
            except ValueError:
                errors.append("Age must be a number")
                suggestions.append("Please provide age as a number")

        return {
            "is_valid": len(errors) == 0,
            "cleaned_data": cleaned_data,
            "errors": errors,
            "suggestions": suggestions
        }

# 3. Data Validation Chain
print("3. DATA VALIDATION CHAIN")
validation_chain = DataValidationChain()

sample_data = {
    "name": "john doe",
    "email": "john.doe@example.com",
    "phone": "555-123-4567",
    "age": "25"
}

result = validation_chain.invoke(sample_data)
print(f"Validation Results: {json.dumps(result, indent=2)}")


## 🔹 6. **StuffDocumentsChain** – Concatenate docs → Single prompt → LLM

> Used when you can fit all documents into a single prompt.

✅ Best for: Small number of short docs.



In [ ]:

from langchain.chains import MapReduceDocumentsChain
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

from langchain_groq import ChatGroq
# Initialize LLM for some chains
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    max_tokens=150,
    timeout=2,
    max_retries=1
)


In [ ]:
stuff_docs = [
    Document(page_content="The sun is a star at the center of our solar system."),
    Document(page_content="It is a massive, hot ball of plasma, held together by gravity."),
    Document(page_content="The sun provides the energy for all life on Earth through sunlight.")
]

print("--- Testing 'stuff' chain ---")
stuff_chain = load_summarize_chain(llm, chain_type="stuff", verbose=True)
result = stuff_chain.invoke(stuff_docs)
print("================Result==========\n")
print(result["output_text"])

--- Testing 'stuff' chain ---


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"The sun is a star at the center of our solar system.

It is a massive, hot ball of plasma, held together by gravity.

The sun provides the energy for all life on Earth through sunlight."


CONCISE SUMMARY:

> Finished chain.

> Finished chain.
================Result==========

The sun is a massive, hot star at the center of our solar system, providing energy for life on Earth through sunlight.



## 🔹 7. **MapReduceChain** – Process chunks and summarize

> Ideal for large documents – breaks into chunks, maps responses, then reduces.

✅ Best for: Long or many documents, chunked processing.



In [ ]:
map_reduce_docs = [
    Document(page_content="**Chapter 1: The Martian Expedition Begins**\n\nThe Ares III mission landed on Mars with a crew of six astronauts, led by Commander Lewis. Their primary objective was to study a specific region of Acidalia Planitia. The initial days were spent deploying habitat modules and setting up scientific instruments. Morale was high, and the early data collected was promising."),
    Document(page_content="**Chapter 2: The Storm and Abandonment**\n\nUnexpectedly, a fierce dust storm, far more powerful than predicted, struck their landing site. The MAV (Mars Ascent Vehicle) was destabilized, and debris was flying everywhere. During the chaotic evacuation, astronaut Mark Watney was struck by flying debris and presumed dead by the crew. Following protocol, Commander Lewis made the agonizing decision to abort the mission and leave Mars without him."),
    Document(page_content="**Chapter 3: Watney's Survival**\n\nMiraculously, Mark Watney survived the storm. A piece of antenna had pierced his suit but sealed itself, preventing depressurization. He was left alone on Mars, with limited supplies and no communication. His immediate challenge was to find a way to survive until rescue could potentially arrive, which seemed impossible. He began to apply his botanical expertise to grow food."),
    Document(page_content="**Chapter 4: Ingenuity and Communication**\n\nWatney, a botanist and mechanical engineer, used his ingenuity to make the habitat habitable, grow potatoes using Martian soil and human waste, and repair the damaged communications equipment. After months of painstaking work, he managed to establish rudimentary communication with NASA, sending a brief, garbled message that confirmed he was alive."),
    Document(page_content="**Chapter 5: The Rescue Mission**\n\nUpon learning of Watney's survival, NASA, along with the international community, launched a desperate and risky rescue mission. The original Ares III crew, already en route back to Earth, volunteered to turn around and retrieve Watney, undertaking an even more dangerous slingshot maneuver around Earth. The world watched with bated breath as the daring plan unfolded."),
    Document(page_content="**Chapter 6: Return to Earth**\n\nAfter a series of nail-biting maneuvers and a dramatic spacewalk, Watney was successfully retrieved by the Ares III crew. He returned to Earth a hero, a symbol of human resilience and the triumph of science and cooperation. The mission's success opened new avenues for future interplanetary travel and highlighted the importance of contingency planning in space exploration.")
]

print("--- Testing 'map_reduce' chain ---")
map_reduce_chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
result = map_reduce_chain.invoke(map_reduce_docs)
print(result["output_text"])
print("\n" + "="*50 + "\n")

--- Testing 'map_reduce' chain ---


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"**Chapter 1: The Martian Expedition Begins**

The Ares III mission landed on Mars with a crew of six astronauts, led by Commander Lewis. Their primary objective was to study a specific region of Acidalia Planitia. The initial days were spent deploying habitat modules and setting up scientific instruments. Morale was high, and the early data collected was promising."


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"**Chapter 2: The Storm and Abandonment**

Unexpectedly, a fierce dust storm, far more powerful than predicted, struck their landing site. The MAV (Mars Ascent Vehicle) was destabilized, and debris was flying everywhere. During the chaotic evacuation, astronaut Mark Watney was struck by flying debris and presumed dead by the crew. Following protocol, 


## 🔹 8. **RefineDocumentsChain** – Incremental summarization with context

> LLM sees initial doc → generates summary → sees next + current summary → refines it.


✅ Best for: Context-sensitive refinement over multiple documents.

In [ ]:
refine_docs = [
    Document(page_content="**Part 1: The Discovery of Penicillin**\n\nIn 1928, Scottish bacteriologist Alexander Fleming made a serendipitous discovery that would revolutionize medicine. While studying Staphylococcus bacteria, he noticed a mold contaminating one of his petri dishes. Around the mold, there was a clear ring where bacterial growth was inhibited. This mold was later identified as Penicillium notatum."),
    Document(page_content="**Part 2: Initial Observations and Challenges**\n\nFleming's initial observations showed that the mold produced a substance capable of killing a wide range of harmful bacteria. He named this active substance 'penicillin.' However, extracting and purifying penicillin proved to be a significant challenge. Fleming himself struggled to isolate it in sufficient quantities for practical medical use."),
    Document(page_content="**Part 3: Florey and Chain's Breakthrough**\n\nIt wasn't until the late 1930s and early 1940s that a team at the University of Oxford, led by Howard Florey and Ernst Chain, took up Fleming's work. They successfully developed methods for mass-producing and purifying penicillin. Their research confirmed its incredible therapeutic potential and opened the door for its widespread use."),
    Document(page_content="**Part 4: Impact on World War II and Beyond**\n\nPenicillin played a crucial role in World War II, saving countless lives by treating bacterial infections in wounded soldiers. Its widespread availability post-war dramatically reduced mortality rates from previously fatal diseases like pneumonia and sepsis. This discovery ushered in the era of antibiotics, transforming modern medicine and public health.")
]

print("--- Testing 'refine' chain ---")
refine_chain = load_summarize_chain(llm, chain_type="refine", verbose=True)
result = refine_chain.invoke(refine_docs)
print(result["output_text"])
print("\n" + "="*50 + "\n")

--- Testing 'refine' chain ---


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"**Part 1: The Discovery of Penicillin**

In 1928, Scottish bacteriologist Alexander Fleming made a serendipitous discovery that would revolutionize medicine. While studying Staphylococcus bacteria, he noticed a mold contaminating one of his petri dishes. Around the mold, there was a clear ring where bacterial growth was inhibited. This mold was later identified as Penicillium notatum."


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: In 1928, Alexander Fleming discovered penicillin when he noticed a mold, later identified as Penicillium notatum, inhibiting bacterial growth around it.
We have the opportunity to refine the existing summary (only if needed) with 

## 🔹 9. **ConversationalRetrievalChain** – RAG-like conversational memory

> Merges **contextual memory** with **document retrieval**.


In [ ]:
!pip install -q -U faiss-cpu langchain_huggingface # faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.1 MB/s eta 0:00:00


In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

try:
    # Use GroqEmbeddings when loading the FAISS index
    retriever = FAISS.load_local("faiss_index", embeddings).as_retriever()
except Exception as e:
    print(f"Could not load FAISS index: {e}. Creating a test_doc.txt one for demonstration.")
    # Create dummy documents with recent AI topics
    with open("test_doc.txt", "w") as f:
        f.write("""
        Generative AI has seen rapid advancements in 2023-2024, with models like GPT-4 and Claude 3 pushing boundaries in language understanding and generation. These models are now capable of creating various forms of content, from text to images and even video, with increasing sophistication.

        The impact of generative AI on the job market is a significant discussion. While some fear job displacement due to automation of routine tasks, many experts also highlight the creation of new roles, particularly in areas like AI development, prompt engineering, and AI ethics. The focus is shifting towards augmenting human capabilities rather than outright replacement, requiring workers to upskill and reskill.

        Ethical considerations in AI, such as bias in models, data privacy, and the responsible deployment of AI systems, are also major ongoing discussions. Developers and policymakers are working on frameworks and regulations to ensure AI is developed and used safely and equitably.
        """)

    loader = TextLoader("test_doc.txt")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # Create a dummy FAISS index using Embeddings
    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local("faiss_index")
    retriever = vectorstore.as_retriever()


Could not load FAISS index: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).. Creating a test_doc.txt one for demonstration.


In [ ]:

from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq

# Initialize LLM for some chains
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    max_tokens=256,
    timeout=2,
    max_retries=1
)


qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever)

chat_history = [
    ("What are the biggest trends in AI right now?", "Generative AI, especially large language models (LLMs) and their applications, is a huge trend."),
    ("How is Generative AI impacting the job market?", "It's automating some tasks but also creating new roles like prompt engineering and AI ethics specialists. The focus is on augmentation and upskilling."),
    ("What are some ethical concerns around AI?", "Bias in models, data privacy, and ensuring responsible and equitable deployment of AI systems are major ethical concerns.")
]

response = qa_chain.invoke({"question": "Why AI systems are major ethical concerns ?", "chat_history": chat_history})
print(response["answer"])


According to the provided context, AI systems are major ethical concerns due to several reasons, including:

1. Bias in models: AI models can perpetuate and amplify existing biases present in the data used to train them, leading to unfair outcomes.
2. Data privacy: AI systems often rely on large amounts of personal data, raising concerns about how this data is collected, stored, and used.
3. Responsible deployment: There are concerns about the safe and equitable deployment of AI systems, ensuring they are used in a way that benefits society as a whole.

These concerns are driving discussions and efforts to develop frameworks and regulations that ensure AI is developed and used responsibly.


In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
response = qa.invoke({"query": "What does the document say about agents?"})
print(response["result"])



I don't know. The document does not mention the term "agents" at all.


## 🔹 10. **DocumentStuffChain** – Multiple doc → Chunked analysis → Question-Answer

✅ Best for: Running analysis on multiple documents with source output.


In [ ]:
from langchain_core.documents import Document
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_groq import ChatGroq

# Initialize LLM for some chains
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    max_tokens=256,
    timeout=2,
    max_retries=2
)


docs = [
    Document(
        page_content="""The IMF's April 2025 World Economic Outlook projects global growth to decline due to escalating trade tensions and policy-induced uncertainty. Global headline inflation is expected to decline but at a slower pace than anticipated, particularly as services inflation remains persistent. Central banks are balancing growth-inflation trade-offs, and fiscal policies need recalibration to ensure debt sustainability. Aging populations also pose challenges to economic growth, while policies promoting healthy aging and increased labor force participation for older individuals and women could help mitigate adverse effects. India is projected to become the fourth-largest economy in 2025, driven by strong private consumption and infrastructure development.""",
        metadata={"source": "IMF World Economic Outlook, April 2025"}
    ),
    Document(
        page_content="""Geopolitical risks continue to significantly impact global supply chains in 2024. Armed conflicts, like those in the Middle East and Eastern Europe, disrupt trade routes (e.g., Red Sea shipping) and create uncertainty. Trade conflicts and proposed tariffs are prompting companies to reassess strategies, leading to potential nearshoring and increased production costs. New regulations and climate change impacts (extreme weather events) also add to the vulnerabilities of logistics and supply networks, necessitating diversified supplier bases and buffer inventories.""",
        metadata={"source": "Maersk Report, Feb 2025; DHL Insights"}
    ),
    Document(
        page_content="""Artificial intelligence (AI) is set to have a "nontrivial, but modest" effect on global GDP in the next decade, with some estimates suggesting a 1% to 1.8% boost. While AI can automate routine tasks and create efficiencies, the profitable application of AI is still limited to about 5% of tasks. It is projected to generate significant value in the global economy by 2030 through increased spending on AI solutions and new revenue streams. However, AI also presents job displacement risks, particularly for white-collar workers with higher education, while roles requiring manual labor or emotional intelligence are less susceptible. The slow widespread adoption means fears of mass job replacement might be overblown for now.""",
        metadata={"source": "MIT Sloan, IDC Research, Dec 2024"}
    ),
    Document(
        page_content="""Geopolitical events continue to heavily influence energy markets. Regions with concentrated oil reserves remain volatile, impacting prices. OPEC decisions on production quotas directly affect global oil supply and costs. Disruptions to maritime routes (chokepoints) due to tensions or piracy can lead to significant price volatility for oil and LNG. Sanctions, like those on Russia, affect energy investment and supply, contributing to price fluctuations. The transition towards sustainable energy is ongoing, but geopolitical factors can complicate the pace and direction of this shift, as nations balance energy security with decarbonization goals.""",
        metadata={"source": "IEA World Energy Outlook 2024, S&P Global"}
    )
]


qa_chain = load_qa_with_sources_chain(llm, chain_type="stuff")


In [ ]:
from langchain.chains import AnalyzeDocumentChain
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0) # Define a splitter
split_docs = text_splitter.split_documents(docs)


# Test with the first document
print("--------------- Test DocumentStuffChain with IMF Outlook --------------")
result_imf = qa_chain.invoke({
    "input_documents": split_docs,
    "question": "Can you predict the possibility of this tariff war ?"
    })

print(f"Question : {result_imf['question']}")
print(f"Answer : {result_imf['output_text']}")

# Test with the second document
print("\n--------------- Test DocumentStuffChain with Geopolitical Supply Chains --------------")
result_supply_chain = qa_chain.invoke({
    "input_documents": split_docs,
    "question": "How the supply and costs of oil will be in future ?"
    })
print(f"Question : {result_supply_chain['question']}")
print(f"Answer : {result_supply_chain['output_text']}")


--------------- Test AnalyzeDocumentChain with IMF Outlook --------------
Question : Can you predict the possibility of this tariff war ?
Answer : I don't know the possibility of this tariff war as it is not mentioned in the provided content.

SOURCES: IMF World Economic Outlook, April 2025; Maersk Report, Feb 2025; DHL Insights; MIT Sloan, IDC Research, Dec 2024; IEA World Energy Outlook 2024, S&P Global

--------------- Test AnalyzeDocumentChain with Geopolitical Supply Chains --------------
Question : How the supply and costs of oil will be in future ?
Answer : It is difficult to predict with certainty how the supply and costs of oil will be in the future. However, based on the provided sources, it can be inferred that geopolitical risks, trade tensions, and policy-induced uncertainty will continue to impact global supply chains and energy markets. The IMF's World Economic Outlook projects a decline in global growth and a slower pace of decline in global headline inflation, while th


## 🔹 11. **ConversationalChain** – Memory + LLM for multi-turn interaction

✅ Best for: Stateful conversations where chat history is preserved.

In [ ]:

# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationChain

# memory = ConversationBufferMemory()
# conv_chain = ConversationChain(llm=llm, memory=memory, verbose=True)

# conv_chain.invoke("Hello, who are you?")
# conv_chain.invoke("What's the weather like?")



from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain, LLMChain
from langchain.prompts import PromptTemplate
# from langchain.vectorstores import FAISS
from langchain.chains.question_answering.chain import load_qa_chain
from langchain.schema import Document
import os

# from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq


# Initialize LLM for some chains
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    max_tokens=256,
    timeout=2,
    max_retries=2
)

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")



memory = ConversationBufferMemory()
conv_chain = ConversationChain(llm=llm, memory=memory, verbose=True)

# List to store chat history (responses from conv_chain.invoke)
chat_history_responses = []

# Initial interactions
response1 = conv_chain.invoke("Hello, who are you?")
chat_history_responses.append(response1['response'])
print(f"User: Hello, who are you?\nAI: {response1['response']}\n")

response2 = conv_chain.invoke("What's the weather like?")
chat_history_responses.append(response2['response'])
print(f"User: What's the weather like?\nAI: {response2['response']}\n")

print("\n--- Starting a meaningful conversation on defense technology ---\n")

# Meaningful conversation on current advancements in defense technology
user_prompt_1 = "Let's shift gears and discuss something more strategic. What are some of the most significant current advancements in defense technology that you've observed?"
response_ml_1 = conv_chain.invoke(user_prompt_1)
chat_history_responses.append(response_ml_1['response'])
print(f"User: {user_prompt_1}\nAI: {response_ml_1['response']}\n")

user_prompt_2 = "That's true, AI seems to be everywhere. What about the physical systems themselves? Are there any game-changers in terms of new weaponry?"
response_ml_2 = conv_chain.invoke(user_prompt_2)
chat_history_responses.append(response_ml_2['response'])
print(f"User: {user_prompt_2}\nAI: {response_ml_2['response']}\n")

user_prompt_3 = "That sounds like a significant threat. And what about the other end of the spectrum, smaller, more networked systems?"
response_ml_3 = conv_chain.invoke(user_prompt_3)
chat_history_responses.append(response_ml_3['response'])
print(f"User: {user_prompt_3}\nAI: {response_ml_3['response']}\n")

user_prompt_4 = "So, it's not just about offense, but also defense against these new threats. What about the less visible aspects of warfare, like in the digital realm?"
response_ml_4 = conv_chain.invoke(user_prompt_4)
chat_history_responses.append(response_ml_4['response'])
print(f"User: {user_prompt_4}\nAI: {response_ml_4['response']}\n")

user_prompt_5 = "It sounds like a constant race between developing new offensive technologies and creating defenses against them."
response_ml_5 = conv_chain.invoke(user_prompt_5)
chat_history_responses.append(response_ml_5['response'])
print(f"User: {user_prompt_5}\nAI: {response_ml_5['response']}\n")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-40-1dc72f92c81f>:37: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-40-1dc72f92c81f>:38: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conv_chain = ConversationChain(llm=llm, memory=memory, verbose=True)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, who are you?
AI:

> Finished chain.
User: Hello, who are you?
AI: Hello there, I'm delighted to make your acquaintance. I'm an artificial intelligence model known as Lumin, and I'm a large language model specifically designed to assist and converse with humans in a friendly and informative manner. I was created by a team of researchers at Meta AI, and I'm based on a transformer architecture that allows me to process and understand human language in a highly efficient and accurate way.

I have been trained on a massive corpus of text data that includes a wide range of topics, from science and history to entertainment and culture. This training data includ

In [ ]:

print("\n--- Q&A from chat history using a QA Chain ---\n")

# Prepare chat history as documents for the QA chain
history_docs = []
for message in memory.buffer_as_messages:
    # Convert message to a string and then to a Document
    # Ensure the content is always a string, even if the model returns something else for 'content'
    content = f"{message.type.capitalize()}: {str(message.content)}"
    history_docs.append(Document(page_content=content))

# Initialize a QA chain using the specified LLM
qa_chain = load_qa_chain(llm, chain_type="stuff")

def answer_question_from_history(question: str) -> str:
    """Answers a question by querying the chat history using the QA chain."""
    result = qa_chain.invoke({"input_documents": history_docs, "question": question})
    return result["output_text"]

# Q&A from chat history
q1 = "What is one of the most impactful advancements in defense technology mentioned in our conversation?"
a1 = answer_question_from_history(q1)
print(f"Q: {q1}\nA: {a1}\n")

q2 = "What is the defining characteristic of hypersonic weapons?"
a2 = answer_question_from_history(q2)
print(f"Q: {q2}\nA: {a2}\n")

q3 = "What is a key advantage of drone swarm technology?"
a3 = answer_question_from_history(q3)
print(f"Q: {q3}\nA: {a3}\n")

q4 = "What kind of infrastructure can be targeted in cyber warfare?"
a4 = answer_question_from_history(q4)
print(f"Q: {q4}\nA: {a4}\n")

q5 = "What does the discussion suggest about the overall nature of modern warfare?"
a5 = answer_question_from_history(q5)
print(f"Q: {q5}\nA: {a5}\n")

print("\n--- Full Chat History (AI Responses Only) ---")
for i, response_text in enumerate(chat_history_responses):
    print(f"Response {i+1}: {response_text}")


--- Q&A from chat history using a QA Chain ---



<ipython-input-41-123d1c1bb0ec>:12: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff")


Q: What is one of the most impactful advancements in defense technology mentioned in our conversation?
A: One of the most impactful advancements in defense technology mentioned in our conversation is the increasing use of artificial intelligence (AI) and machine learning in defense systems. This includes the development of AI-powered surveillance systems, such as the Advanced Battle Management System (ABMS), which uses machine learning algorithms to analyze vast amounts of data from various sources, including sensors, drones, and satellites.

This advancement has the potential to significantly improve the effectiveness of defense systems by enabling them to quickly process and analyze large amounts of data, identify patterns and threats, and make decisions in real-time. This could lead to improved situational awareness, faster response times, and more effective countermeasures against enemy threats.

Q: What is the defining characteristic of hypersonic weapons?
A: The defining characte

## 🔹 12. **LLMCheckerChain** – Verification & correction loop

> Use an LLM to **check another LLM’s output**, and refine if needed.

✅ Best for: Safety checks or multi-step validation using LLMs.


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.chains import LLMChain
from langchain.chains import LLMCheckerChain
from langchain_groq import ChatGroq


# Initialize LLM for some chains
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.5,
    max_tokens=1024,
    timeout=5,
    max_retries=2
)

# Define the initial translation chain using LCEL
translate_prompt = PromptTemplate.from_template("Translate '{sentence}' to Bengali")
translate_chain = translate_prompt | llm | StrOutputParser()

# Define a checking prompt
check_prompt = PromptTemplate.from_template("""
You are a language checker. Given an original English sentence and a proposed Bengali translation,
evaluate if the translation is accurate and grammatically correct.
If it is accurate, just output "ACCURATE".
If it is not accurate or incorrect, explain why and provide a corrected translation in Bengali.

Original English Sentence: {original_sentence}
Proposed Bengali Translation: {translated_sentence}

Evaluation:
""")

# Define the checking chain
check_chain = check_prompt | llm | StrOutputParser()

# Combine the translation and checking chains using LCEL
# This runnable first translates the sentence and then uses the output
# along with the original sentence for the checking chain.
full_checking_process = RunnableParallel(
    original_sentence=RunnablePassthrough(), # Pass the input sentence as original_sentence
    translated_sentence=translate_chain # Run the translation chain
) | check_chain # Pipe the output of the parallel block into the check chain

print("--- Testing Translation and Checking Chain ---")

# Invoke the combined process
sentence_to_check = "A drone swarm technology has the ability to overwhelm and saturate enemy air defenses through sheer numbers"
result = full_checking_process.invoke(sentence_to_check)

print(f"Original Sentence: {sentence_to_check}")
print(f"Checking Result:\n{result}")

--- Testing Translation and Checking Chain ---
Original Sentence: A drone swarm technology has the ability to overwhelm and saturate enemy air defenses through sheer numbers
Checking Result:
The proposed Bengali translation is not entirely accurate. 

The original English sentence uses the phrase "overwhelm and saturate" which implies a continuous and intense attack. However, the proposed translation uses the phrase "সম্পূর্ণরূপে নষ্ট করে দেওয়ার" which means "completely destroy" which is more like a one-time action.

A more accurate translation would be: ড্রোন সুইম প্রযুক্তির ক্ষমতা আছে বিপজ্জনক শত্রু বিমান ব্যবস্থাপনা সুরক্ষা ভেঙে দেওয়ার এবং সংখ্যার মাধ্যমে তা সমৃদ্ধ করে দেওয়ার।

However, a more idiomatic translation would be: ড্রোন সুইম প্রযুক্তির ক্ষমতা আছে বিপজ্জনক শত্রু বিমান ব্যবস্থাপনা সুরক্ষাকে ভেঙে দিতে এবং সংখ্যার মাধ্যমে তাদের সুরক্ষাকে অতিমাত্রায় সমৃদ্ধ করে দিতে।

Note: অতিমাত্রায় সমৃদ্ধ করে দিতে is a more idiomatic translation of "saturate" in this context.


## 🔹 13. **Self-AskWithSearchChain** – Ask → Plan → Search → Answer



In [ ]:
!pip install -q -U langchain-tavily

In [ ]:

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults


# Initialize LLM for some chains
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3,
    max_tokens=256,
    timeout=2,
    max_retries=3
)

# Use Tavily Search Results tool instead of Google Search
search = TavilySearchResults()
tools = [search]

# Create the prompt for the agent
# This prompt guides the LLM to decide when and how to use the search tool
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. You have access to a search tool to find information."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("=== SEARCH-AUGMENTED AGENT TEST ===")

# Invoke the agent with the question
# Note: The input variable is typically 'input' for agents
result = agent_executor.invoke({"input": "Give the % change of all sectorial index performace in Indian stock market on today?"})

print("\n============Result==========================\n")
print(result["output"])

=== SEARCH-AUGMENTED AGENT TEST ===


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Indian stock market sectorial index performance today % change'}`


[{'title': 'All NSE Sectoral Indices in India', 'url': 'https://www.etmoney.com/stocks/market-data/sectoral-indices/32', 'content': 'Indices | Price | 1D Change % | 1W Change % | 1Y Change % | 3Y Change %\nNifty Auto | 23,763.15 | +1.05 | -1.20 | -0.54 | +114.14\nNifty Bank | 55,572.00 | +0.31 | +0.27 | +13.48 | +58.35\nNifty Commodities | 8,764.85 | +0.46 | -0.23 | -4.12 | +62.16\nNifty CPSE | 6,537.70 | +0.11 | +0.31 | -3.49 | +156.27\nNifty Energy | 35,882.60 | +0.69 | +0.54 | -13.82 | +39.29\nNifty FMCG | 57,049.70 | +0.97 | +0.17 | +2.89 | +50.73\nNifty IT | 37,785.90 | +1.02 | +0.82 | +11.71 | +35.28 [...] Nifty Media | 1,691.50 | +0.50 | +1.19 | -10.83 | -11.49\nNifty Metal | 9,336.45 | +0.94 | +1.36 | -5.96 | +79.04\nNifty MNC | 28,187.15 | +0.44 | -0.20 | -3.33 | +58.47\nNifty Ph

## 🔹 14. **LCEL (LangChain Expression Language)** – Declarative Chain Composition

> **NEW, recommended** for composability, clarity, and tight control.



✅ Best for: Clean, Pythonic pipelines with intermediate state access.

In [ ]:


from langchain.schema.runnable import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_together import Together

llm = Together(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=0.2,
    max_tokens=512,
    # timeout=2,
    # max_retries=1,
)

# # Initialize LLM for some chains
# llm = ChatGroq(
#     model="llama-3.1-8b-instant",
#     temperature=0.7,
#     max_tokens=1024,
#     timeout=6,
#     max_retries=3
# )


chain = (
    {"topic": RunnablePassthrough()}
    | PromptTemplate.from_template("Write a sanskrit shloka about {topic} from Mahabharat time period")
    | llm
)

result = chain.invoke("time travel")
print(result)



Here's a Sanskrit shloka that might fit the bill:

"कालचक्रे परिभ्रमन्, युगानां परिवर्तनम् |
दृश्यन्ते भूतपूर्वाणि, भविष्याणि च पश्यन् ||"

Transliteration: "Kālacakra paribhraman, yugānāṁ parivartanam |
Dṛśyante bhūtapūrvāṇi, bhaviṣyāṇi ca paśyan ||"

Translation: "As one travels through the cycle of time, witnessing the transformation of ages |
One sees the past and the future, and beholds the events that have been and those that will be ||"

This shloka is inspired by the style of the Mahabharata, but please note that it is not an actual quote from the epic. The language and meter are intended to evoke the spirit of ancient Sanskrit poetry.

In this shloka, the phrase "कालचक्रे परिभ्रमन्" (Kālacakra paribhraman) suggests the idea of traveling through the cycle of time, while "युगानां परिवर्तनम्" (yugānāṁ parivartanam) refers to the transformation of ages or epochs. The second line, "दृश्यन्ते भूतपूर्वाणि, भविष्याणि च पश्यन्" (Dṛśyante bhūtapūrvāṇi, bhaviṣyāṇi ca paśyan), implies th




## 🔹 15. **SQLDatabaseChain** – Query SQL DB using natural language

✅ Best for: Natural language to SQL querying with LLM.



In [4]:
!pip install -q -U  sqlalchemy ipython-sql langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00


In [6]:
import sqlite3
import os

db_file = "mydb.db"

# Remove existing DB to start fresh
if os.path.exists(db_file):
    os.remove(db_file)

# Create and populate the database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create a 'users' table
cursor.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    signup_date DATE NOT NULL
);
''')

# Insert dummy data
cursor.execute("INSERT INTO users (username, signup_date) VALUES ('Alice', '2025-05-20')")
cursor.execute("INSERT INTO users (username, signup_date) VALUES ('Bob', '2025-05-21')")
cursor.execute("INSERT INTO users (username, signup_date) VALUES ('Charlie', '2025-05-22')")
cursor.execute("INSERT INTO users (username, signup_date) VALUES ('David', '2025-05-23')")
cursor.execute("INSERT INTO users (username, signup_date) VALUES ('Eve', '2025-05-10')")  # Old user
cursor.execute("INSERT INTO users (username, signup_date) VALUES ('Frank', '2025-05-24')")
cursor.execute("INSERT INTO users (username, signup_date) VALUES ('Grace', '2025-05-25')")

conn.commit()
conn.close()
print(f"Database '{db_file}' created and populated successfully.\n")

Database 'mydb.db' created and populated successfully.



In [ ]:
# # To inspect the tables (optional, using ipython-sql for convenience)
# %load_ext sql
# %sql sqlite:///mydb.db

# %config SqlMagic.style='DEFAULT' # Use a supported style instead


# %sql SELECT * FROM users;

In [9]:
import os
from langchain.prompts import PromptTemplate
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_together import Together
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.2,
    max_tokens=256,
    timeout=2,
    max_retries=2
)

# llm = Together(
#     model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
#     temperature=0.2,
#     max_tokens=1024,
#     # timeout=2,
#     # max_retries=1,
# )


# Load the database using SQLAlchemy-style URI
db = SQLDatabase.from_uri(f"sqlite:///{db_file}")

# Improved custom prompt to handle fixed date correctly
custom_prompt = PromptTemplate.from_template("""
You are a SQLite expert. Given a question, write a syntactically correct SQL query for SQLite.

- Do not include any Markdown formatting like triple backticks (```).
- Just return the raw SQL query — nothing else.
- Assume today's date is '2025-05-27'.

{table_info}

Question: {input}
SQLQuery:
""")

# Initialize the SQLDatabaseChain
sql_chain = SQLDatabaseChain.from_llm(
    llm=llm,
    db=db,
    prompt=custom_prompt,
    verbose=True,
    return_intermediate_steps=True,
    use_query_checker=False  # Disable to avoid rewriting correct date logic
)


# 1. Count users signed up last week
print("Query 1: Users signed up last week (based on 2025-05-27):")
result1 = sql_chain.invoke({"query": "How many users signed up last week? (Assume today is 2025-05-27)"})
print(f"Generated SQL: {result1['intermediate_steps']}")
print(f"Answer: {result1['result']}")

# 2. List users who signed up today
print("\nQuery 2: Users who signed up today (2025-05-27):")
result2 = sql_chain.invoke({"query": "List all users who signed up today. (Assume today is 2025-05-27)"})
print(f"Generated SQL: {result2['intermediate_steps']}")
print(f"Answer: {result2['result']}")

# 3. Signups per day for the past 7 days
print("\nQuery 3: Daily signups for the past 7 days:")
result3 = sql_chain.invoke({"query": "How many users signed up on each day in the last 7 days? (Assume today is 2025-05-27)"})
print(f"Generated SQL: {result3['intermediate_steps']}")
print(f"Answer: {result3['result']}")

Query 1: Users signed up last week (based on 2025-05-27):


> Entering new SQLDatabaseChain chain...
How many users signed up last week? (Assume today is 2025-05-27)
SQLQuery:SELECT COUNT(*) 
FROM users 
WHERE signup_date >= DATE('2025-05-20') AND signup_date <= DATE('2025-05-27')
SQLResult: [(6,)]
Answer:SELECT COUNT(*) 
FROM users 
WHERE signup_date >= DATE('2025-05-20') AND signup_date <= DATE('2025-05-27')
> Finished chain.
Generated SQL: [{'input': 'How many users signed up last week? (Assume today is 2025-05-27)\nSQLQuery:', 'top_k': '5', 'dialect': 'sqlite', 'table_info': '\nCREATE TABLE users (\n\tid INTEGER, \n\tusername TEXT NOT NULL, \n\tsignup_date DATE NOT NULL, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from users table:\nid\tusername\tsignup_date\n1\tAlice\t2025-05-20\n2\tBob\t2025-05-21\n3\tCharlie\t2025-05-22\n*/', 'stop': ['\nSQLResult:']}, "SELECT COUNT(*) \nFROM users \nWHERE signup_date >= DATE('2025-05-20') AND signup_date <= DATE('2025-05-27')", {'sql_cmd': "SELECT CO



## 🧠 Pro Tips

* Use **LangChain Expression Language (LCEL)** for composable chains in a more readable format.
* You can add **callbacks/logging** with `verbose=True` or via `LangchainTracer`.
* All chains return a dictionary by default. Use `.run()` for string output (when supported).
* For **streaming responses**, pass `streaming=True` to the LLM and set a callback handler.





## ✅ **LangChain Chains Comparison Table**

| Chain Name                         | Use Case                         | Level of Control | Memory Efficient? | Best When...                                        |
| ---------------------------------- | -------------------------------- | ---------------- | ----------------- | --------------------------------------------------- |
| `LLMChain`                         | Basic prompt → response          | 🔹 High          | ✅ Yes             | You want simple LLM prompt/response interaction     |
| `SequentialChain`                  | Multi-step deterministic process | 🔹🔹 Very High   | ✅ Yes             | Steps have fixed order and dependent outputs        |
| `RouterChain` / `MultiPromptChain` | Route to specialized chains      | 🔹🔹 Very High   | ✅ Yes             | Inputs need dynamic routing based on topic/intent   |
| `StuffDocumentsChain`              | All docs → one input             | 🔸 Low           | ❌ No              | All text fits in one prompt                         |
| `MapReduceDocumentsChain`          | Large docs → partial + merge     | 🔹 Medium        | ✅ Yes             | Text is large or needs chunked summarization        |
| `RefineDocumentsChain`             | Iterative improvement            | 🔹 Medium        | ✅ Yes             | Each new doc should modify prior output             |
| `ConversationalChain`              | Stateful LLM convo               | 🔹 Medium        | ✅ Yes             | Chat history must persist                           |
| `TransformChain`                   | Preprocessing / postprocessing   | 🔹🔹 Very High   | ✅ Yes             | You need full control over I/O shaping              |
| `SelfAskWithSearchChain`           | Answer via planning + search     | 🔹 Medium        | ✅ Yes             | Web-enabled reasoning with multiple hops            |
| `SQLDatabaseChain`                 | Natural language → SQL           | 🔹 Medium        | ✅ Yes             | Use LLMs to query tabular/SQL data                  |
| `LCEL`                             | Declarative, composable pipeline | 🔹🔹🔹 Highest   | ✅ Yes             | You want clean functional-style LLM flow definition |

---

## 📊 Flow Diagram: **When to Use Which LangChain Chain**

```plaintext
                 ┌────────────────────┐
                 │ What is your goal?│
                 └─────────┬──────────┘
                           │
        ┌──────────────────┼───────────────────┐
        ▼                                      ▼
Simple Prompt/Response                Document Processing
   (Single-step LLM)                  (Chunked or Long)
        │                                      │
        ▼                                      ▼
  Use `LLMChain`                  ┌────────────┼────────────┐
                                  ▼                         ▼
                        Docs fit in prompt?       Docs too long?
                              │                         │
                         Yes ▼                         ▼ Yes
                        Use `Stuff`             Use `MapReduce` or `Refine`

        ┌──────────────────┐
        │ Multi-step Logic │────────────┐
        └───────┬──────────┘            │
                ▼                       ▼
     Use `SequentialChain`       Dynamic Routing Needed?
                                       │
                                  Yes ▼
                            Use `RouterChain`

        ┌──────────────────┐
        │ Stateful Dialogue│
        └───────┬──────────┘
                ▼
         Use `ConversationalChain`

        ┌──────────────────┐
        │ External Knowledge? (search/sql)│
        └───────┬──────────┘
                ▼
  Use `SelfAskWithSearch` or `SQLDatabaseChain`

        ┌────────────────────────────┐
        │ Want complete I/O control? │
        └───────┬────────────────────┘
                ▼
        Use `TransformChain` or `LCEL`

```

---

## 🎯 Recommendations by Use Case

| Use Case                     | Recommended Chain(s)                              |
| ---------------------------- | ------------------------------------------------- |
| Simple Prompt Completion     | `LLMChain`, `LCEL`                                |
| Multi-step Reasoning Flow    | `SequentialChain`, `TransformChain`, `LCEL`       |
| Long Doc Summarization       | `MapReduceDocumentsChain`, `RefineDocumentsChain` |
| Domain-based Prompt Routing  | `RouterChain`, `MultiPromptChain`                 |
| Conversational Agents        | `ConversationChain` with `ConversationMemory`     |
| Data Lookup or Search        | `SelfAskWithSearchChain`, `SQLDatabaseChain`      |
| High Modularity / Debuggable | `LCEL`, `TransformChain`, `SequentialChain`       |

---

